# Note

- Please view in darkmode. 

- This kernel is inspired by Susan Li. Check her publications at https://towardsdatascience.com/@actsusanli

In [ ]:
import numpy as np
import pandas as pd 

import seaborn as sns 
import matplotlib.dates as md
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import host_subplot
import mpl_toolkits.axisartist as AA
plt.style.use(['fivethirtyeight', 'dark_background'])

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from mpl_toolkits.mplot3d import Axes3D

!pip install pyemma
from pyemma import msm
%matplotlib inline

import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train = pd.read_csv('/kaggle/input/expedia-personalized-sort/data/train.csv')

In [ ]:
train.head()

In [ ]:
train.isnull().sum()

# Subset data
Select property / visitor location country / srch_room_count with the most data points 

In [ ]:
# prop_id corresponding to 
train['prop_id'].value_counts()

In [ ]:
train['visitor_location_country_id'].value_counts()

In [ ]:
# Num of rooms specified in search by customer
train['srch_room_count'].value_counts()

In [ ]:
# Subset df 
df = train.loc[train['prop_id'] == 104517]

df = df.loc[df['visitor_location_country_id'] == 219]

df = df.loc[df['srch_room_count'] == 1]

# srch_saturday = if stay includes Sat night 
# srch_booking_window = num of days between search date and hotel stay start date 
df = df[['date_time', 'price_usd', 'srch_booking_window', 'srch_saturday_night_bool']]

In [ ]:
df.info()

In [ ]:
df.describe()

Point anomaly = max price usd 5584

In [ ]:
train.loc[(train['price_usd'] == 5584) & 
         (train['visitor_location_country_id'] == 219)]

Possible wrong search, no intention to book

In [ ]:
# Remove 5584 
df = df.loc[df['price_usd'] < 5584]
df['price_usd'].describe()

In [ ]:
df['date_time'].min(), df['date_time'].max()

In [ ]:
df['date_time'].describe()

df['date_time'] = pd.to_datetime(df['date_time'])

df.head()

In [ ]:
df.plot(x = 'date_time', 
        y = 'price_usd', 
        figsize = (16, 8))

plt.xlabel('dates')
plt.ylabel('USD')
plt.title('Time series of room price by date of search');

In [ ]:
df.head()

In [ ]:
a = df.loc[df['srch_saturday_night_bool'] == 0, 'price_usd']
b = df.loc[df['srch_saturday_night_bool'] == 1, 'price_usd']

plt.figure(figsize = (16, 8))

plt.hist(a, bins = 80, 
         alpha = 0.3, 
         label = 'search w/o Sat night stay')

plt.hist(b, bins = 80, 
         alpha = 0.3, 
         label = 'search w/ Sat night stay')

plt.xlabel('Price')
plt.ylabel('Freq')
plt.legend()
plt.title('Sat night search')
plt.plot();

In [ ]:
df['srch_saturday_night_bool'].value_counts()

In [ ]:
print('Kurtosis: %f' % df['price_usd'].kurt())
print('Skewness: %f' % df['price_usd'].skew())

In [ ]:
sns.distplot(df['price_usd'], 
                 hist = False, label = 'USD')

sns.distplot(df['srch_booking_window'], 
                  hist = False, label = 'booking window')

plt.xlabel('dist')
sns.despine()

In [ ]:
sns.distplot(a, hist = False, rug = False)
sns.distplot(b, hist = False, rug = False)

sns.despine()

In [ ]:
df = df.sort_values('date_time')
df['date_time_int'] = df.date_time.astype(np.int64)

# Cluster-based models

- k-means
- isolation forest
- clustering

Potential outliers:
- usd
- srch_booking_window (days between search and first stay date)
- srch_saturday (stay includes sat night

# K-means

- create 'k' similar clusters of instances  
- Instances outside of clusters = possible anomalies 

In [ ]:
# Determine optimal cluster num using elbow method 
data = df[['price_usd', 'srch_booking_window', 'srch_saturday_night_bool']]
n_cluster = range(1, 20)

kmeans = [KMeans(n_clusters = i).fit(data) for i in n_cluster]
scores = [kmeans[i].score(data) for i in range(len(kmeans))]

In [ ]:
# elbow curve 
fig, ax = plt.subplots(figsize = (16, 8))
ax.plot(n_cluster, scores, color = 'orange')

plt.xlabel('clusters num')
plt.ylabel('score')
plt.title('elbow curve')
plt.show();

set n_clusters to 7

n_clusters > 7 = additional clusters do not explain greater variance in variable 

where variable = price_usd

In [ ]:
# k means output 
X = df[['price_usd', 'srch_booking_window', 'srch_saturday_night_bool']]
X = X.reset_index(drop = True)

km = KMeans(n_clusters = 7)
km.fit(X)
km.predict(X)

labels = km.labels_

X.head()

# 3D clusters 
plot using k means output 

In [ ]:
fig = plt.figure(1, figsize = (7, 7))

ax = Axes3D(fig, rect = [0, 0, 0.95, 1], 
            elev = 48, azim = 134)

ax.scatter(X.iloc[:, 0], 
           X.iloc[:, 1], 
           X.iloc[:, 2],
           c = labels.astype(np.float), edgecolor = 'm')

ax.set_xlabel('USD')
ax.set_ylabel('srch_booking_window')
ax.set_zlabel('srch_saturday_night_bool')

plt.title('K Means', fontsize = 10);

In [ ]:
import pylab as pl 

# PCA 

In [ ]:
#Y = df[['price_usd']]
#X = df[['srch_booking_window']]

#Nc = range(1, 20)
#kmeans = [KMeans(n_clusters = i) for i in Nc]

#score = [kmeans[i].fit(Y).score(Y) for i in range(len(kmeans))]

#plt.figure(figsize = (16, 8))
#pl.plot(Nc, score)
#pl.xlabel('cluster num')
#pl.ylabel('score')
#pl.title('elbow curve')
#pl.show();

In [ ]:
#pca = PCA(n_components = 1).fit(Y)

#pca_d = pca.transform(Y)
#pca_c = pca.transform(X)

#kmeans = KMeans(n_clusters = 7)
#kmeansoutput = kmeans.fit(Y)

#pl.figure('7 cluster k-means')
#pl.figure(figsize = (16, 8))

#pl.scatter(pca_c[:, 0], 
#           pca_d[:, 0], 
#           c = kmeansoutput.labels_)

#pl.xlabel('booking window')
#pl.ylabel('USD')
#pl.title('7 cluster')
#pl.show();

In [ ]:
data = df[['price_usd', 'srch_booking_window', 'srch_saturday_night_bool']]

X = data.values
X_std = StandardScaler().fit_transform(X)

# Calc eigenvec cor & eig_vals of covar matrix 
mean_vec = np.mean(X_std, axis = 0)

cov_mat = np.cov(X_std.T)

eig_vals, eig_vecs = np.linalg.eig(cov_mat)

# eig_val,eig_vecs tuple
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

eig_pairs.sort(key = lambda x: x[0], reverse = True)

In [ ]:
# Calc explained var from eig_vals 
total = sum(eig_vals)

# Individual explained var 
var_exp = [(i/total)*100 for i in sorted(eig_vals, reverse = True)]

# Cumulative explained var 
cum_var_exp = np.cumsum(var_exp)

In [ ]:
plt.figure(figsize = (16, 8))
plt.bar(range(len(var_exp)), var_exp, 
        alpha = 0.5, align = 'center', 
        label = 'individual explained var', 
        color = 'r'
       )

plt.step(range(len(cum_var_exp)), cum_var_exp,
         where = 'mid',
         label = 'cumulative explained var')

plt.xlabel('principal components')
plt.ylabel('explained var ratio')
plt.legend(loc = 'best')
plt.show();

Component 1 explains approx 50% of var. 
2 = explains < 40
3 = explains < 20 

Components 1 + 2 = explain approx 80% of var 

- set n_components = 2
- standardize features 

In [ ]:
data = df[['price_usd', 'srch_booking_window', 'srch_saturday_night_bool']]

# Standardize features
X_std = StandardScaler().fit_transform(X)
data = pd.DataFrame(X_std)

# Reduce components to 2 
pca = PCA(n_components = 2)
data = pca.fit_transform(data)

# Standardize 2 new features 
scaler = StandardScaler()
np_scaled = scaler.fit_transform(data)
data = pd.DataFrame(np_scaled)

In [ ]:
kmeans = [KMeans(n_clusters = i).fit(data) for i in n_cluster]

df['cluster'] = kmeans[7].predict(data)
df.index = data.index

df['pc1'] = data[0]
df['pc2'] = data[1]
df['cluster'].value_counts()

In [ ]:
def getDistanceByPoint(data, model):
    distance = pd.Series()
    for i in range(0,len(data)):
        Xa = np.array(data.loc[i])
        Xb = model.cluster_centers_[model.labels_[i]-1]
        distance.set_value(i, np.linalg.norm(Xa-Xb))
    return distance

In [ ]:
outliers_fraction = 0.01

distance = getDistanceByPoint(data, kmeans[9])
outlier_num = int(outliers_fraction * len(distance))

threshold = distance.nlargest(outlier_num).min()

df['anomaly'] = (distance >= threshold).astype(int)

In [ ]:
fig, ax = plt.subplots(figsize = (12, 6))

colors = {0:'blue', 1:'red'}

ax.scatter(df['pc1'], df['pc2'], 
           c = df['anomaly'].apply(lambda x: colors[x]))

plt.xlabel('pc1')
plt.ylabel('pc2')
plt.show();

In [ ]:
df = df.sort_values('date_time')
df['date_time'] = df.date_time.astype(np.int64)

# object with anomalies
a = df.loc[df['anomaly'] == 1, 
           ['date_time_int', 'price_usd']]

a

In [ ]:
fig, ax = plt.subplots(figsize = (10, 5))

ax.plot(df['date_time_int'], df['price_usd'], 
        color = 'orange', label = 'Normal')

ax.scatter(a['date_time_int'], a['price_usd'],
           color = 'red', label = 'Anomaly')

plt.xlabel('time')
plt.ylabel('USD')
plt.legend()
plt.show();


In [ ]:
df['anomaly'].unique()

In [ ]:
a = df.loc[df['anomaly'] == 0, 'price_usd']
b = df.loc[df['anomaly'] == 1, 'price_usd']

fig, axs = plt.subplots(figsize = (10, 5))
axs.hist([a, b], 
         bins = 50, stacked = True, 
         color = ['orange', 'red'])

plt.show();

In [ ]:
df.anomaly.value_counts()

In [ ]:
df.head()

# Isolation Forest

Detect anomalies based on data points that are few and different 

- No use of density / distance measure 
    i.e. different from clustering based / distanced based algorithms 

- Randomly select a feature 
    
- Randomly select a split between max and min values of selected feature 
    
- Length of path, avged over a forest of random trees = measure of normality 

- Random partitioning = shorter path for anomalies

- If forest produces shorter paths for samples, then they are likely to be anomalies 

In [ ]:
data = df[['price_usd', 'srch_booking_window', 'srch_saturday_night_bool']]

scaler = StandardScaler()
np_scaled = scaler.fit_transform(data)

data = pd.DataFrame(np_scaled)

# Isolation forest 
outliers_fraction = 0.01
ifo = IsolationForest(contamination = outliers_fraction)

ifo.fit(data)

df['anomaly1'] = pd.Series(ifo.predict(data))

fig, ax = plt.subplots(figsize = (10, 5))

a = df.loc[df['anomaly1'] == -1, ['date_time_int', 'price_usd']]

ax.plot(df['date_time_int'], df['price_usd'], 
        color = 'orange', label = 'Normal')

ax.scatter(a['date_time_int'], a['price_usd'], 
           color = 'red', label = 'Anomaly')

plt.legend()
plt.show();

In [ ]:
df['anomaly1'].unique()

In [ ]:
a = df.loc[df['anomaly1'] == 1, 'price_usd']
b = df.loc[df['anomaly1'] == -1, 'price_usd']

fig, ax = plt.subplots(figsize = (10, 5))

ax.hist([a, b],
        bins = 50, stacked = True, 
        color = ['orange', 'red'] )

plt.show();

In [ ]:
df['anomaly1'].value_counts()

# Support vector machine models

Associated with supervised learning 

- One class SVM
- Gaussian dist 
- Markov chain

# One class SVM 
- unsupervised anomaly detection 
- estimate support of high dimensional distribution

In [ ]:
data = df[['price_usd', 'srch_booking_window', 'srch_saturday_night_bool']]
scaler = StandardScaler()
np_scaled = scaler.fit_transform(data)

data = pd.DataFrame(np_scaled)

# Train 

osvm = OneClassSVM(nu = outliers_fraction, 
                   kernel = 'rbf', 
                   gamma = 0.01)

osvm.fit(data)

df['anomaly2'] = pd.Series(osvm.predict(data))

In [ ]:
fig, ax = plt.subplots(figsize = (10, 5))

a = df.loc[df['anomaly2'] == -1, 
           ['date_time_int', 'price_usd']]

ax.plot(df['date_time_int'], df['price_usd'], 
        color = 'orange', 
        label = 'Normal')

ax.scatter(a['date_time_int'], a['price_usd'], 
           color = 'red', 
           label = 'Anomaly')

plt.legend()
plt.show();

In [ ]:
df.head()

In [ ]:
a = df.loc[df['anomaly2'] == 1, 'price_usd']
b = df.loc[df['anomaly2'] == -1, 'price_usd']

fig, ax = plt.subplots(figsize = (10, 5))

ax.hist([a, b], bins = 50, 
        stacked = True, color = ['orange','red'])

plt.show();

In [ ]:
df['anomaly2'].value_counts()

# Gaussian distribution

- Assume data is normally distributed

- Use covariance.EllipticEnvelope from scikit-learn to find key params of general distribution by assuming entire dataset = an expression of an underlying multivariate Gaussian distribution


Create two dfs based on categories defined by sat boolean

In [ ]:
df_class0 = df.loc[df['srch_saturday_night_bool'] == 0, 'price_usd']
df_class1 = df.loc[df['srch_saturday_night_bool'] == 1, 'price_usd']

fig, axs = plt.subplots(1, 2)

df_class0.hist(ax = axs[0], bins = 50, color = 'orange')
df_class1.hist(ax = axs[1], bins = 50, color = 'red');

Apply EllipticEnvelope to each category 

Set contamination param (proportion of outliers present in dataset)

Use decision function to compute decision function of given observations (equivalent to shifted Mahalanobis distances. 

Threshold for identifying as outliers = 0 (compatible with other detection algorithms)

predict(x_train) predict labels of X_train according to fitted model

1 = normal
-1 = anomaly

In [ ]:
envelope = EllipticEnvelope(contamination = outliers_fraction)

x_train = df_class0.values.reshape(-1, 1)
envelope.fit(x_train)

df_class0 = pd.DataFrame(df_class0)
df_class0['deviation'] = envelope.decision_function(x_train)
df_class0['anomaly'] = envelope.predict(x_train)

In [ ]:
envelope = EllipticEnvelope(contamination = outliers_fraction)

x_train = df_class1.values.reshape(-1, 1)
envelope.fit(x_train)

df_class1 = pd.DataFrame(df_class1)
df_class1['deviation'] = envelope.decision_function(x_train)
df_class1['anomaly'] = envelope.predict(x_train)

In [ ]:
df_class = pd.concat([df_class0, df_class1])
df['anomaly3'] = df_class['anomaly']

fig, ax = plt.subplots(figsize = (10, 5))

a = df.loc[df['anomaly3'] == -1, 
           ('date_time_int', 'price_usd')]

ax.plot(df['date_time_int'], df['price_usd'], 
        color = 'orange')

ax.scatter(a['date_time_int'], a['price_usd'],
          color = 'red')

plt.show();

In [ ]:
df['anomaly3'].value_counts()

In [ ]:
a = df.loc[df['anomaly3'] == 1, 'price_usd']
b = df.loc[df['anomaly3'] == -1, 'price_usd']

fig, ax = plt.subplots(figsize = (10, 5))
ax.hist([a, b], 
        bins = 50, stacked = True, 
        color = ['orange', 'red'])

plt.show();

# Results 
anomalies detected only show abnormally high prices, no abnormally low prices 